In [ ]:
import json
import openai
from dotenv import load_dotenv
import os
load_dotenv()



from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json
from utils.db import get_table_columns, get_connection,list_schemas,list_tables # <-- make sure this exists in your utils.db



In [22]:
src_schema = 'stg'
src_table = 'brand_master'
tgt_schema = 'int'
tgt_table = 'brand_master'



# Fetch schema metadata from DB
source_columns = get_table_columns(src_schema, src_table)
target_columns = get_table_columns(tgt_schema, tgt_table)

# Extract just the column names for prompt clarity
src_cols = [c["column_name"] for c in source_columns]
tgt_cols = [c["column_name"] for c in target_columns]

# Include schema overview for context
all_schemas = {s: list_tables(s) for s in list_schemas()}

# Build LLM model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2,api_key=os.getenv('OPENAI_API_KEY'))

# Strict instruction to use only target columns
messages = [
    SystemMessage(content=(
        "You are an expert ETL engineer. "
        "Your task is to build a JSON Source-To-Target Mapping (STTM). "
        "You must only use the target columns that exist in the provided schema metadata. "
        "Do not invent or hallucinate new target columns. "
        "You may, however, infer transformations or use constants for audit fields like timestamps."
        "Include join details if the column comes from another schema/table."
        "Use the key 'join_logic' and 'lookup_table' for such cases."

    )),
    HumanMessage(content=f"""
    Given:
    - Primary source table: `{src_schema}.{src_table}`
    - Target table: `{tgt_schema}.{tgt_table}`

    Available schemas and tables:
    {json.dumps(all_schemas, indent=2)}

    Source columns:
    {json.dumps(src_cols, indent=2)}

    Target columns:
    {json.dumps(tgt_cols, indent=2)}

    Generate an STTM JSON with entries like this for simple table:
    {{
        "target_column": "<column_name_in_target>",
        "source_columns": ["col1", "col2", ...],
        "transformation": "<expression or null>",
        "pkey" : "<Primary Keys for the target table>"
    }}

    Generate an STTM JSON with entries like this for complex table that need joins:
    
    {{
        "target_column": "product_name",
        "pkey" : "<Primary Keys for the target table>"
        "source_tables": [
      {{
        "schema": "ref",
        "table": "product_master",
        "join_type": "inner",
        "join_logic": "stg.sales_txn.product_id = ref.product_master.product_id"
      }}
      ],
        "source_columns": ["product_id"],
        "transformation": "<expression or null>"
        
        
    }}
    

    Rules:
    - Map only columns present in the target list.
    - Use similar names for direct mappings (case-insensitive) except for audit columns.
    - For audit fields like 'audit_insrt_dt', 'audit_updt_dt', 'load_ts', use CURRENT_TIMESTAMP.
    - If you use more than one table, include the "source_tables" field in the mapping like this:
    - If join is needed, specify both lookup_table and join_logic
    - Output must be valid JSON (list of mappings).
    -INT schema tables are always SCD1 for dimensions
    -DWH Schema tables are always SCD2 for dimensions.
    -Process_ind in the INT Schema will be I for new records & U for updated records & D for deleted records.
    -When the source schema is INT and target schema is DWH pick up records which have processind in I and U.
    -When looking up records from DWH always use the filter IS_ACTIVE='Y'
    -When looking up records from INT always use the filter PROCESS_IND in ('I','U')
    -DWH tables will have is_active column which is a flag .Its set to Y for currently active records and N for inactive records.
    -EFF_START_DT & EFF_END_DT are the dates in between which the records were active/currently active.
    -Provide the Primary_key details as well by profiling if needed.

    """)
]

response = model.invoke(messages)
reply = response.content.strip()

# Safe JSON extraction
try:
    sttm = json.loads(reply)
except Exception:
    import re
    match = re.search(r"\[[\s\S]*\]", reply)
    sttm = json.loads(match.group(0)) if match else []

# Double-check target column validity (safety filter)
valid_tgt_cols = set(tgt_cols)
filtered_sttm = [m for m in sttm if m.get("target_column") in valid_tgt_cols]

print(filtered_sttm)

c:\Users\A4627\OneDrive - Axtria\DSA_Python\Agentic_DataIngestion\utils\db.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
c:\Users\A4627\OneDrive - Axtria\DSA_Python\Agentic_DataIngestion\utils\db.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
c:\Users\A4627\OneDrive - Axtria\DSA_Python\Agentic_DataIngestion\utils\db.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
c:\Users\A4627\OneDrive - Axtria\DSA_Python\Agentic_Dat

[{'target_column': 'brand_id', 'source_columns': ['brand_id'], 'transformation': None, 'pkey': 'brand_id'}, {'target_column': 'brand_name', 'source_columns': ['brand_name'], 'transformation': None, 'pkey': 'brand_id'}, {'target_column': 'process_ind', 'source_columns': [], 'transformation': "CASE WHEN EXISTS (SELECT 1 FROM int.brand_master WHERE brand_id = stg.brand_master.brand_id) THEN 'U' ELSE 'I' END", 'pkey': 'brand_id'}, {'target_column': 'audit_insrt_dt', 'source_columns': [], 'transformation': 'CURRENT_TIMESTAMP', 'pkey': 'brand_id'}, {'target_column': 'audit_insrt_nm', 'source_columns': [], 'transformation': 'CURRENT_USER', 'pkey': 'brand_id'}, {'target_column': 'audit_updt_dt', 'source_columns': [], 'transformation': 'CURRENT_TIMESTAMP', 'pkey': 'brand_id'}, {'target_column': 'audit_updt_nm', 'source_columns': [], 'transformation': 'CURRENT_USER', 'pkey': 'brand_id'}, {'target_column': 'audit_batch_id', 'source_columns': [], 'transformation': 'NULL', 'pkey': 'brand_id'}]


In [19]:
reply 


'```json\n[\n    {\n        "target_column": "profit_center_id",\n        "source_columns": ["profit_center_id"],\n        "transformation": null\n    },\n    {\n        "target_column": "profit_center_code",\n        "source_columns": ["profit_center_code"],\n        "transformation": null\n    },\n    {\n        "target_column": "profit_center_name",\n        "source_columns": ["profit_center_name"],\n        "transformation": null\n    },\n    {\n        "target_column": "parent_profit_center_id",\n        "source_columns": ["parent_profit_center_id"],\n        "transformation": null\n    },\n    {\n        "target_column": "brand_id",\n        "source_columns": ["brand_id"],\n        "transformation": null\n    },\n    {\n        "target_column": "level",\n        "source_columns": ["level"],\n        "transformation": null\n    },\n    {\n        "target_column": "brand_name",\n        "source_tables": [\n            {\n                "schema": "dwh",\n                "table": "b

In [ ]:
source_columns

In [ ]:
df=get_table_columns('int', '')
df